In [3]:
import os
import pickle as pkl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
import pydeseq2
from pydeseq2.dds import DeseqDataSet
from pydeseq2.default_inference import DefaultInference
from pydeseq2.ds import DeseqStats

In [7]:
from pydeseq2.utils import build_design_matrix
from deconveil.dds import deconveil_fit
from deconveil.inference import Inference
from deconveil.default_inference import DefInference
from deconveil.utils_CNaware import *
from deconveil import deconveil_fit
from deconveil.ds import deconveil_stats

In [9]:
def run_pydeseq2(rna_counts, metadata, output_path, design_factors="condition", alpha=0.05):
    """
    Runs PyDESeq2 analysis and saves the results.

    Parameters:
        rna_counts (pd.DataFrame): Count matrix with genes as rows and samples as columns.
        metadata (pd.DataFrame): Metadata for the samples with design factors.
        output_path (str): Directory to save the results.
        design_factors (str): Column in metadata to use for design.
        alpha (float): Significance level for statistical tests.
    """
    os.makedirs(output_path, exist_ok=True)
    
    # Initialize DESeq2 analysis
    inference = DefaultInference(n_cpus=8)
    dds = DeseqDataSet(
        counts=rna_counts,
        metadata=metadata,
        design_factors="condition",
        refit_cooks=True,
        inference=inference,
    )

    # Fit DESeq2 model
    dds.fit_size_factors()
    dds.fit_genewise_dispersions()
    dds.fit_dispersion_trend()
    dds.fit_dispersion_prior()
    dds.fit_MAP_dispersions()
    dds.fit_LFC()
    dds.calculate_cooks()
    
    if dds.refit_cooks:
        dds.refit()

    # Perform statistical analysis
    stat_res_pydeseq = DeseqStats(dds, alpha=alpha, cooks_filter=True, independent_filter=True)
    stat_res_pydeseq.run_wald_test()

    if stat_res_pydeseq.cooks_filter:
        stat_res_pydeseq._cooks_filtering()
    stat_res_pydeseq.p_values

    if stat_res_pydeseq.independent_filter:
        stat_res_pydeseq._independent_filtering()
    else:
        stat_res_pydeseq._p_value_adjustment()

    # Log-fold change shrinkage
    stat_res_pydeseq.lfc_shrink(coeff="condition_B_vs_A")
    stat_res_pydeseq.summary()

    # Save results
    results_path = os.path.join(output_path, f"{replicate_num}_res_CNnaive_{n_samples}_{n_genes}.csv")
    stat_res_pydeseq.results_df.to_csv(results_path)
    return(stat_res_pydeseq.results_df)


def run_deconveil(rna_counts, metadata, cnv, output_path, design_factors="condition", alpha=0.05):
    """
    Runs DeConveil analysis and saves the results.

    Parameters:
        rna_counts (pd.DataFrame): Count matrix with genes as rows and samples as columns.
        metadata (pd.DataFrame): Metadata for the samples with design factors.
        cnv (pd.DataFrame): Copy number variation (CNV) data matrix  with genes as rows and samples as columns.
        output_path (str): Directory to save the results.
        design_factors (str): Column in metadata to use for design.
        alpha (float): Significance level for statistical tests.
    """
    os.makedirs(output_path, exist_ok=True)
    
    # Initialize DeConveil inference
    inference = DefInference(n_cpus=8)

    # Fit DeConveil model
    dds = deconveil_fit(
        counts=rna_counts,
        metadata=metadata,
        cnv=cnv,
        design_factors=design_factors,
        inference=inference,
        refit_cooks=True
    )
    dds.fit_size_factors()
    dds.fit_genewise_dispersions()
    dds.fit_dispersion_trend()
    dds.fit_dispersion_prior()
    dds.fit_MAP_dispersions()
    dds.fit_LFC()
    dds.calculate_cooks()

    if dds.refit_cooks:
        dds.refit()  # Replace outlier counts

    # Statistical analysis
    stat_res_deconveil = deconveil_stats(
        dds, 
        alpha=alpha, 
        independent_filter=True, 
        cooks_filter=True
    )
    stat_res_deconveil.run_wald_test()

    if stat_res_deconveil.independent_filter:
        stat_res_deconveil._independent_filtering()
    else:
        stat_res_deconveil._p_value_adjustment()

    # Log-fold change shrinkage
    stat_res_deconveil.lfc_shrink(coeff="condition_B_vs_A")
    stat_res_deconveil.summary()

    # Save results
    results_path = os.path.join(output_path, f"{replicate_num}_res_CNaware_{n_samples}_{n_genes}.csv")
    stat_res_deconveil.results_df.to_csv(results_path)
    return(stat_res_deconveil.results_df)

In [23]:
sample_sizes = [10,20,40,100]
gene_counts = [3000]
#num_replicates = 10
noise_levels = [0, 0.05, 0.10, 0.15, 0.20]

In [25]:
def process_and_save_replicates(replicate_num, n_samples, n_genes, noise_level):
    try:
        noise_label = f"noise_{int(noise_level * 100)}" if noise_level > 0 else "no_noise"
        base_path = f"/Users/katsiarynadavydzenka/Documents/PhD_AI/deconveilCaseStudies/simulations/data/simulations_3/{noise_label}/"

        # Updated filenames to match your actual format
        rna_path = os.path.join(base_path, f"{replicate_num}_{n_samples}_{n_genes}_rna_cn.csv")
        cnv_path = os.path.join(base_path, f"{replicate_num}_{n_samples}_{n_genes}_cn.csv")
        metadata_path = os.path.join(base_path, f"{replicate_num}_{n_samples}_{n_genes}_metadata.csv")

        # Load data
        rna = pd.read_csv(rna_path, index_col=0).T
        cnv = pd.read_csv(cnv_path, index_col=0).T
        cnv = (cnv * 2).astype(int)
        metadata = pd.read_csv(metadata_path, index_col=0)

        # Output directories
        output_base = "/Users/katsiarynadavydzenka/Documents/PhD_AI/deconveilCaseStudies/simulations/results/simulation_3"
        pydeseq_output_path = os.path.join(output_base, f"{noise_label}/replicates_pydeseq/")
        deconveil_output_path = os.path.join(output_base, f"{noise_label}/replicates_deconveil/")
        os.makedirs(pydeseq_output_path, exist_ok=True)
        os.makedirs(deconveil_output_path, exist_ok=True)

        # Run tools
        res_pydeseq = run_pydeseq2(rna, metadata, pydeseq_output_path)
        res_deconveil = run_deconveil(rna, metadata, cnv, deconveil_output_path)

        print(f"✅ Processed replicate {replicate_num} | Samples: {n_samples}, Genes: {n_genes}, Noise: {noise_label}")
    except Exception as e:
        print(f"Error in replicate {replicate_num} | Samples: {n_samples}, Genes: {n_genes}, Noise: {noise_label}: {e}")


In [27]:
for n_samples in sample_sizes:
    for n_genes in gene_counts:
        for noise in noise_levels:
            for replicate_num in range(1, num_replicates + 1):
                process_and_save_replicates(replicate_num, n_samples, n_genes, noise)

Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.21 seconds.

Fitting dispersion trend curve...
... done in 0.03 seconds.

Fitting MAP dispersions...
... done in 0.25 seconds.

Fitting LFCs...
... done in 0.08 seconds.

Calculating cook's distance...
... done in 0.00 seconds.

Replacing 12 outlier genes.

Fitting dispersions...
... done in 0.01 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.01 seconds.

Running Wald tests...
... done in 0.07 seconds.

Fitting MAP LFCs...
... done in 0.17 seconds.



Log2 fold change & Wald test p-value: condition B vs A
           baseMean  log2FoldChange     lfcSE      stat    pvalue      padj
g1     5.586918e+02        0.824020  0.414512  2.535765  0.011220  0.055001
g2     2.917013e+03        0.537544  0.354804  1.992277  0.046341  0.145859
g3     1.860974e+06        2.071686  0.179824  0.883106  0.377179  0.547430
g4     4.111789e+03        0.483675  0.320625  1.806502  0.070840  0.189919
g5     1.927870e+04       -0.270198  0.679262 -0.648863  0.516427  0.668358
...             ...             ...       ...       ...       ...       ...
g996   9.627871e+02       -0.896217  0.380297 -2.880491  0.003971  0.025112
g997   4.768346e+03       -0.381224  0.435037 -0.994111  0.320169  0.497156
g998   9.999611e+02       -0.758858  0.378645 -2.538804  0.011123  0.054794
g999   6.715092e+02       -0.481316  0.312096 -1.890419  0.058702  0.169170
g1000  2.107611e+02       -0.691291  0.399958 -2.253841  0.024206  0.093460

[1000 rows x 6 columns]


Fitting size factors...
... done in 0.00 seconds.

Fitting dispersions...
... done in 0.13 seconds.

Fitting dispersion trend curve...
... done in 0.03 seconds.

Fitting MAP dispersions...
... done in 0.20 seconds.

Fitting LFCs...
... done in 0.12 seconds.

Calculating cook's distance...
... done in 0.00 seconds.

Replacing 8 outlier genes.

Fitting dispersions...
... done in 0.00 seconds.

Fitting MAP dispersions...
... done in 0.01 seconds.

Fitting LFCs...
... done in 0.01 seconds.

Running Wald tests...


replace_mask before filtering: (20, 8)
Number of True values in replace_mask: 10
replacement_counts_trimmed shape: (7, 8)


... done in 0.06 seconds.

Fitting MAP LFCs...
... done in 0.13 seconds.



...

Fitting dispersions...
... done in 0.25 seconds.

Fitting dispersion trend curve...
... done in 0.04 seconds.

Fitting MAP dispersions...
... done in 0.25 seconds.

Fitting LFCs...
... done in 0.15 seconds.

Calculating cook's distance...
... done in 0.03 seconds.

Replacing 89 outlier genes.



replace_mask before filtering: (200, 89)
Number of True values in replace_mask: 91
replacement_counts_trimmed shape: (47, 89)


Fitting dispersions...
... done in 0.04 seconds.

Fitting MAP dispersions...
... done in 0.04 seconds.

Fitting LFCs...
... done in 0.04 seconds.

Running Wald tests...
... done in 0.15 seconds.

Fitting MAP LFCs...


Log2 fold change & Wald test p-value: condition B vs A
           baseMean  log2FoldChange     lfcSE       stat        pvalue  \
g1     5.622894e+02        1.588864  0.123863  10.098340  5.618497e-24   
g2     2.414184e+03        0.832557  0.103702   4.221499  2.426825e-05   
g3     1.970392e+06        2.796805  0.045026   5.875742  4.209530e-09   
g4     4.192420e+03        2.099743  0.124194  11.972076  4.976780e-33   
g5     2.300411e+04        2.201167  0.290163   3.073626  2.114743e-03   
...             ...             ...       ...        ...           ...   
g996   9.027935e+02        0.310138  0.095341  -2.591519  9.555318e-03   
g997   4.452861e+03       -0.541572  0.137387  -5.048534  4.452123e-07   
g998   7.705826e+02        0.372882  0.119564  -0.777598  4.368058e-01   
g999   7.590674e+02       -0.712192  0.109302  -8.301903  1.024567e-16   
g1000  2.228154e+02        0.813202  0.125034   1.608547  1.077155e-01   

               padj  
g1     5.508331e-23  
g2     6.112

... done in 0.19 seconds.

